In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# Function to calculate RSI
def calculate_rsi(data, window=14):
    delta = data['Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

# Function to calculate MACD
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    short_ema = data['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = data['Close'].ewm(span=long_window, adjust=False).mean()
    
    macd = short_ema - long_ema
    signal_line = macd.ewm(span=signal_window, adjust=False).mean()
    
    return macd, signal_line

def fetch_and_analyze_etf_data(ticker):
    asset = yf.Ticker(ticker)
    end_date = datetime.today().strftime('%Y-%m-%d')
    start_date = (datetime.today() - timedelta(days=730)).strftime('%Y-%m-%d')
    data = asset.history(start=start_date, end=end_date)
    data.reset_index(inplace=True)
    data['rsi'] = calculate_rsi(data)
    data['macd'], data['signal_line'] = calculate_macd(data)
    return data[['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'rsi', 'macd', 'signal_line']]

etf_data = fetch_and_analyze_etf_data("EWJ")
etf_data.to_csv("etf_momentum_analysis.csv", index=False)


In [2]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

def fetch_index_data(tickers):
    all_data = []
    for ticker, name in tickers.items():
        asset = yf.Ticker(ticker)
        end_date = datetime.today().strftime('%Y-%m-%d')
        start_date = (datetime.today() - timedelta(days=730)).strftime('%Y-%m-%d')
        data = asset.history(start=start_date, end=end_date)
        data.reset_index(inplace=True)
        data['index_name'] = name
        all_data.append(data)
    return pd.concat(all_data, ignore_index=True)

tickers = {
    "^N225": "Nikkei 225",
    "^JPN": "Japan Index"
}

index_data = fetch_index_data(tickers)
index_data.to_csv("japan_index_prices.csv", index=False)
